In [6]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import toolkitETL
import importlib
importlib.reload(toolkitETL)

<module 'toolkitETL' from 'd:\\Pedro\\Mestrado\\ETL-ML-Dengue\\src\\etl\\toolkitETL.py'>

In [7]:
## Estabelecendo diretórios base
diretorio_src_etl = os.getcwd()
diretorio_src = Path(diretorio_src_etl).parent.absolute()
diretorio_base = Path(diretorio_src).parent.absolute()
diretorio_base = "F:\\Mestrado_pedro"
diretorio_data = os.path.join(diretorio_base, 'data')

# Bronze
diretorio_bronze = os.path.join(diretorio_data, 'bronze')
diretorio_cnes_pf_bronze = os.path.join(diretorio_bronze, 'cnes_pf')
diretorio_cnes_pf_bronze_csv = os.path.join(diretorio_cnes_pf_bronze, 'csv')

# Silver
diretorio_silver = os.path.join(diretorio_data, 'silver')
diretorio_cnes_pf_silver = os.path.join(diretorio_silver, 'cnes_pf')
diretorio_cnes_pf_silver_tsv = os.path.join(diretorio_cnes_pf_silver, 'tsv')

## Criação de pastas para armazenar os arquivos
os.makedirs(os.path.join(diretorio_data), exist_ok=True)
os.makedirs(os.path.join(diretorio_silver), exist_ok=True)
os.makedirs(os.path.join(diretorio_cnes_pf_silver), exist_ok=True)
os.makedirs(os.path.join(diretorio_cnes_pf_silver_tsv), exist_ok=True)




In [8]:
arquivos_csv = [arquivo for arquivo in os.listdir(diretorio_cnes_pf_bronze_csv) if arquivo.endswith("csv")]
arquivos_csv = arquivos_csv[-5:]
arquivos_tsv = [arquivo.rstrip("csv") + "tsv" for arquivo in arquivos_csv]
arquivos_tsv

['2010_1_2010_12_AC_CNES-PF.tsv', '2020_1_2020_12_AC_CNES-PF.tsv']

In [9]:
colunas_importantes_drop_dupli = ["CNES", "CODUFMUN", "CPF_PROF", "CBO", "VINCUL_C", "VINCUL_A", 
                       "VINCUL_N", "PROF_SUS", "PROFNSUS", "HORAOUTR", "HORAHOSP", "HORA_AMB",
                       "COMPETEN", "nome"
]

colunas_importantes = ["CNES", "CODUFMUN", "VINCUL_C", "VINCUL_A", 
                       "VINCUL_N", "PROF_SUS", "PROFNSUS", "HORAOUTR", "HORAHOSP", "HORA_AMB",
                       "COMPETEN", "nome"
]

colunas_decodificar = {
    'VINCUL_C':
    {1: "Sim",
    0: "Não"},
    'VINCUL_A':
    {1: "Sim",
    0: "Não"},
    'VINCUL_N':
    {1: "Sim",
    0: "Não"},
    'PROF_SUS':
    {1: "Sim",
    0: "Não"},
    'PROFNSUS':
    {1: "Sim",
    0: "Não"},
}

column_map = {
    "nome": "nome_cargo"
}

columns_to_convert = []

In [10]:
for arquivo_csv, arquivo_tsv in zip(arquivos_csv, arquivos_tsv):
    df = pd.read_csv(os.path.join(diretorio_cnes_pf_bronze_csv, arquivo_csv), sep=',')

    for coluna in colunas_importantes_drop_dupli:
        if coluna not in df.columns:
            df[coluna] = np.nan
            continue
    df = df[colunas_importantes_drop_dupli]
    df = df.drop_duplicates()
    df = df[colunas_importantes]


    for coluna in colunas_decodificar:
        df[coluna] = df[coluna].apply(lambda x: toolkitETL.aplicar_decoficador(x, coluna, colunas_decodificar))
    df.replace("NA", np.nan, inplace=True)
    df.replace("", np.nan, inplace=True)
    df = df.applymap(lambda x: x.strip('"') if isinstance(x, str) else x)
    df['NU_MES'] = df['COMPETEN'].apply(lambda x: str(x)[-2:].lstrip("0"))
    df['NU_ANO'] = df['COMPETEN'].apply(lambda x: str(x)[:4])
    
    df.columns = df.columns.str.lower()

    df.to_csv(os.path.join(diretorio_cnes_pf_silver_tsv, arquivo_tsv), sep='\t', index=False)

In [4]:
diretorio_dotenv = os.path.join(diretorio_base, ".env")

cursor, conn = toolkitETL.conectar_banco(diretorio_dotenv)
for arquivo_tsv in arquivos_tsv:
    
    caminho_arquivo = os.path.join(diretorio_cnes_pf_silver_tsv, arquivo_tsv)
    toolkitETL.inserir_no_banco(cursor, conn, caminho_arquivo, 'cnes_pf', delimiter='\t',
                                        diretorio_dotenv=diretorio_dotenv)
